In [1]:
import pandas as pd
import numpy as np
import pickle
import model_function
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV
from sklearn.svm import SVR, LinearSVR
import lightgbm as lgb
from gplearn import genetic
import time

In [2]:
with open('./data/X_train_filled_cleaned.pkl', 'rb') as f:
    X_train_filled_cleaned = pickle.load(f)
with open('./data/X_valid_filled_cleaned.pkl', 'rb') as f:
    X_valid_filled_cleaned = pickle.load(f)
with open('./data/X_test_filled_cleaned.pkl', 'rb') as f:
    X_test_filled_cleaned = pickle.load(f)
with open('./data/X_train_unfilled_cleaned.pkl', 'rb') as f:
    X_train_unfilled_cleaned = pickle.load(f)
with open('./data/X_valid_unfilled_cleaned.pkl', 'rb') as f:
    X_valid_unfilled_cleaned = pickle.load(f)  
with open('./data/X_test_unfilled_cleaned.pkl', 'rb') as f:
    X_test_unfilled_cleaned = pickle.load(f) 
with open('./data/X_train_filled_selected.pkl', 'rb') as f:
    X_train_filled_selected = pickle.load(f)
with open('./data/X_valid_filled_selected.pkl', 'rb') as f:
    X_valid_filled_selected = pickle.load(f)
with open('./data/X_test_filled_selected.pkl', 'rb') as f:
    X_test_filled_selected = pickle.load(f)
with open('./data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('./data/y_valid.pkl', 'rb') as f:
    y_valid = pickle.load(f)   

lasso模型，由于其有特征选择能力，所以不使用选择过特征的数据。

In [4]:
start_time = datetime.now()
print("开始时间为:", start_time)

# 创建 LassoCV 模型
lasso_cv_model = LassoCV(cv=5, n_jobs=-1)

# 在训练集上拟合模型
lasso_cv_model.fit(X_train_filled_cleaned.iloc[:, 2:], y_train)

# 输出最佳的 alpha 参数值
best_alpha = lasso_cv_model.alpha_
print("Best alpha:", best_alpha)

# 在验证集上评估模型
lasso_rankic, lasso_valid_df = model_function.evaluate(lasso_cv_model, X_valid_filled_cleaned, y_valid)

# 预测测试集
lasso_test_result = model_function.perform_pred(lasso_cv_model, X_test_filled_cleaned)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-11 18:41:01.411787
Best alpha: 0.0005502479684026031
rankic均值为： 0.06127259637875911
mse on validation: 0.7339562120769548
结束时间为: 2024-04-11 18:41:45.795082
处理时间为： 0:00:44.383295


In [6]:
with open('./data/lasso.pkl', 'wb') as f:
    pickle.dump(lasso_cv_model, f)
with open('./data/lasso_test_result.pkl', 'wb') as f:
    pickle.dump(lasso_test_result, f)

In [5]:
lasso_rankic

,date,RankIC
0,1532,0.090291
1,1533,0.036443
2,1534,0.101295
3,1535,0.103172
4,1536,0.222240
...,...,...
165,1697,0.019589
166,1698,0.066048
167,1699,0.147741
168,1700,-0.000569


In [7]:
lasso_valid_df

,code,date,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,...,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,y,y_pred
0,s_3762,1532,-0.049363,-0.327876,-0.184470,0.957538,0.514535,-0.178804,0.262615,0.142865,...,-5.656422,0.022838,0.255175,0.466691,-0.008425,1.165561,-0.908380,0.316876,0.062471,0.004634
1,s_1105,1532,-0.830488,-0.311627,0.584069,-1.278562,-0.460536,-0.435773,-0.099144,-0.178450,...,-0.001773,0.270062,-0.609777,-0.258974,0.765287,-0.092965,2.047059,-0.587591,0.292565,0.051521
2,s_485,1532,-0.611967,-0.479543,0.358415,-0.725612,0.569907,-0.134128,-0.188719,0.197253,...,-0.001773,0.285909,-0.364142,-0.518479,-0.123197,0.251242,-0.144447,0.023806,-0.266904,0.050679
3,s_5192,1532,-0.454909,-0.374062,-0.047344,-1.239518,-1.725912,-1.540365,-0.137786,-0.526057,...,-0.001773,-0.113225,-0.550655,-0.356903,-0.324723,0.957955,-0.152394,-1.636843,0.129677,0.016726
4,s_4504,1532,-0.172490,0.224101,-0.523753,0.790655,-0.631866,-1.008463,0.100600,0.041595,...,-0.001773,-0.617732,0.929818,1.490273,-1.534488,-0.855457,-0.312162,-0.650463,-0.612287,-0.084510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598360,s_1214,1701,-0.511380,-0.036597,-0.671859,-1.105909,0.070877,-0.133814,0.249865,-0.328357,...,0.004543,7.102314,-0.538366,-0.396312,2.725462,-0.232657,-1.279138,-0.366515,0.011645,0.022671
598361,s_480,1701,-0.860716,-0.300311,-0.047953,0.265122,-0.208310,-0.362190,0.200475,-0.231677,...,0.004543,0.065422,-0.495128,-0.386530,1.220281,-0.559064,-0.575675,-0.664569,-0.356514,-0.006801
598362,s_2221,1701,-0.746822,-0.322084,0.402402,-0.962964,-0.013369,-0.142823,-1.258924,-0.051712,...,0.004543,0.111561,-0.478180,-0.048248,0.658736,-0.253607,1.819920,-0.591091,-0.249570,-0.018162
598363,s_686,1701,1.026631,0.052221,0.439027,-0.940320,0.809852,1.189064,2.470436,-0.636803,...,0.004543,0.827442,-0.496009,-0.224186,0.061485,-0.142140,0.378383,0.999117,-0.037312,0.039807


In [8]:
lasso_test_result

,code,date,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,...,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,y_pred
0,s_4394,1702,0.068291,-0.432540,0.022628,3.165354,0.907793,0.339218,-1.300476,-0.681366,...,0.007007,-0.001962,-0.088637,5.123355,-0.333052,1.406415,0.730259,1.143091,0.428422,-0.158928
1,s_4390,1702,0.422512,-0.440728,0.410388,0.067812,0.818190,0.738554,2.079457,0.064696,...,0.007007,-0.001962,7.090391,0.312029,0.924242,2.686006,0.920996,1.269212,0.789571,0.090498
2,s_2394,1702,0.838988,-0.199078,-0.103651,-0.223763,-0.108430,0.779757,1.270638,0.508430,...,0.007007,-0.001962,-0.022319,-0.051760,-0.309964,-0.664588,0.394401,-0.972088,0.948195,0.011593
3,s_5135,1702,3.305397,1.641047,0.578268,-1.187674,-0.990240,3.947727,-0.211763,-0.102795,...,0.007007,-0.001962,-0.130295,-0.359713,7.704144,0.019878,-1.426578,0.316132,3.116464,-0.102635
4,s_4733,1702,1.017056,0.760480,-1.023568,-0.508261,-2.677140,-0.596081,-0.555019,-0.645279,...,0.007007,-0.001962,-0.230536,-0.072508,0.801403,-0.096708,-3.142005,-0.429479,-0.211853,-0.126513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3908946,s_3445,2606,2.775059,-0.106231,-0.772367,-1.675571,2.633371,3.623154,-0.713525,-0.087002,...,0.013767,0.001375,0.111419,-0.440882,0.754534,0.983301,6.052376,-0.268740,3.766029,0.197180
3908947,s_1559,2606,-0.409325,-0.355444,0.231227,-1.047911,0.239075,-0.186017,-1.139380,1.153228,...,0.013767,0.001375,0.223317,-0.500254,-0.422076,0.425886,-0.823689,0.683850,-0.095134,-0.037326
3908948,s_3627,2606,0.908406,-0.231084,-0.206256,-0.792178,0.989572,1.452187,2.028437,-0.216644,...,0.013767,0.001375,-0.865135,-0.293487,-0.359932,-0.955121,0.268850,-0.232571,1.335906,0.034723
3908949,s_975,2606,0.583221,-0.313091,-1.038109,0.545349,0.918309,0.301894,-0.630959,-0.773183,...,0.013767,0.001375,-0.171120,-0.264226,-0.601279,-0.448135,-0.075550,-0.792248,0.679176,-0.021004


SVR模型，使用填充了缺失值的数据，对经过和未经特征选择的数据各训练一个模型。

In [9]:
start_time = datetime.now()
print("开始时间为:", start_time)

'''
# 定义超参数搜索范围
param_grid = {
    'nu': [0.1, 0.3, 0.5, 0.7, 0.9],  # NuSVR参数nu的候选值
    'C': [0.1, 0.5, 1, 2],  # NuSVR参数C的候选值
    'gamma': ['scale', 'auto', 0.1, 0.3]  # NuSVR参数gamma的候选值
}

# 创建NuSVR模型
nu_svr = NuSVR(kernel='rbf', cache_size=200, max_iter=2000)

# 使用GridSearchCV进行超参数搜索
grid_search = GridSearchCV(nu_svr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
svr_model_filled = grid_search.fit(X_train_filled_cleaned.iloc[:, 2:], y_train)

# 输出最佳超参数组合
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
'''

# 设定模型参数并训练
svr_filled_model = SVR(C=2, kernel='linear', cache_size=300, max_iter=1000)
svr_filled_model.fit(X_train_filled_cleaned.iloc[:, 2:], y_train)

# 在验证集上评估模型
svr_filled_rankic, svr_filled_valid_df = model_function.evaluate(svr_filled_model, X_valid_filled_cleaned, y_valid)

# 预测测试集
svr_filled_test_result = model_function.perform_pred(svr_filled_model, X_test_filled_cleaned)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-11 18:54:58.512333


F:\anaconda\envs\dongwu\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


rankic均值为： -0.005641997454398926
mse on validation: 8.001284938035974
结束时间为: 2024-04-11 19:00:34.838963
处理时间为： 0:05:36.326630


In [10]:
with open('./data/svr_filled.pkl', 'wb') as f:
    pickle.dump(svr_filled_model, f)
with open('./data/svr_filled_test_result.pkl', 'wb') as f:
    pickle.dump(svr_filled_test_result, f)

In [11]:
svr_filled_rankic

,date,RankIC
0,1532,-0.147567
1,1533,-0.112523
2,1534,0.024230
3,1535,0.097111
4,1536,0.140044
...,...,...
165,1697,-0.150506
166,1698,0.005620
167,1699,0.043211
168,1700,0.183819


In [12]:
start_time = datetime.now()
print("开始时间为:", start_time)

svr_selected_model = SVR(C=2, kernel='linear', cache_size=300, max_iter=1000)
svr_selected_model.fit(X_train_filled_selected.iloc[:, 2:], y_train)

# 在验证集上评估模型
svr_selected_rankic, svr_selected_valid_df = model_function.evaluate(svr_selected_model, X_valid_filled_selected, y_valid)

# 预测测试集
svr_selected_test_result = model_function.perform_pred(svr_selected_model, X_test_filled_selected)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-11 19:00:35.657146


F:\anaconda\envs\dongwu\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


rankic均值为： 0.02412963585818248
mse on validation: 4.4256965375786175
结束时间为: 2024-04-11 19:05:38.904803
处理时间为： 0:05:03.247657


In [13]:
with open('./data/svr_selected.pkl', 'wb') as f:
    pickle.dump(svr_selected_model, f)
with open('./data/svr_selected_test_result.pkl', 'wb') as f:
    pickle.dump(svr_selected_test_result, f)

In [14]:
svr_selected_rankic

,date,RankIC
0,1532,-0.008001
1,1533,-0.088285
2,1534,-0.102526
3,1535,-0.121983
4,1536,0.086973
...,...,...
165,1697,0.018195
166,1698,0.059515
167,1699,0.140601
168,1700,-0.138740


In [24]:
start_time = datetime.now()
print("开始时间为:", start_time)

lsvr_selected_model = LinearSVR(C=1, epsilon=0.05, max_iter=1000)
lsvr_selected_model.fit(X_train_filled_selected.iloc[:, 2:], y_train)

# 在验证集上评估模型
lsvr_selected_rankic, lsvr_selected_valid_df = model_function.evaluate(lsvr_selected_model, X_valid_filled_selected, y_valid)

# 预测测试集
lsvr_selected_test_result = model_function.perform_pred(lsvr_selected_model, X_test_filled_selected)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-11 19:47:34.388578


F:\anaconda\envs\dongwu\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
F:\anaconda\envs\dongwu\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


rankic均值为： 0.06835635715373864
mse on validation: 0.7437867529685133
结束时间为: 2024-04-11 19:59:12.958370
处理时间为： 0:11:38.569792


In [25]:
with open('./data/lsvr_selected.pkl', 'wb') as f:
    pickle.dump(lsvr_selected_model, f)
with open('./data/lsvr_selected_test_result.pkl', 'wb') as f:
    pickle.dump(lsvr_selected_test_result, f)

In [26]:
lsvr_selected_rankic

,date,RankIC
0,1532,0.197608
1,1533,0.092854
2,1534,0.169696
3,1535,0.151459
4,1536,0.137898
...,...,...
165,1697,0.133333
166,1698,0.066516
167,1699,0.102632
168,1700,-0.001189


受笔记本性能所限，SVR网格调参难以在本地实现。尝试使用几个参数组合训练，发现训练速度很慢，只好暂且搁置更大范围的调参。用经过特征选择的数据得到的结果比未经特征选择的更好。用LinearSVR这个实现得到的预测效果好于SVR，也略好于Lasso。

LightGBM模型，使用未填充缺失值的数据，填充了缺失值的数据以及经过特征选择的数据各试一遍。

In [36]:
start_time = datetime.now()
print("开始时间为:", start_time)

# 定义LightGBM回归器
lgb_regressor = lgb.LGBMRegressor(force_col_wise=True)

# 定义超参数空间
param_grid = {
    'num_leaves': [63], 
    'max_depth': [10],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.03, 0.05, 0.1]
}

# 定义网格搜索对象
lgb_unfilled_grid_search = GridSearchCV(estimator=lgb_regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# 进行超参数搜索
lgb_unfilled_model = lgb_unfilled_grid_search.fit(X_train_unfilled_cleaned.iloc[:, 2:], y_train)

# 输出最佳超参数组合和得分
print("Best Parameters:", lgb_unfilled_grid_search.best_params_)
print("Best Score:", lgb_unfilled_grid_search.best_score_)

# 在验证集上评估模型
lgb_unfilled_rankic, lgb_unfilled_valid_df = model_function.evaluate(lgb_unfilled_model, X_valid_unfilled_cleaned, y_valid)

# 预测测试集
lgb_unfilled_test_result = model_function.perform_pred(lgb_unfilled_model, X_test_unfilled_cleaned)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-11 20:31:16.075379
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.018577
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.015366
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.011142
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.013043
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257532, number of used features: 19
[LightGBM] [Info] Start training from score -0.016644
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info]

In [37]:
with open('./data/lgb_unfilled.pkl', 'wb') as f:
    pickle.dump(lgb_unfilled_model, f)
with open('./data/lgb_unfilled_test_result.pkl', 'wb') as f:
    pickle.dump(lgb_unfilled_test_result, f)

In [38]:
lgb_unfilled_rankic

,date,RankIC
0,1532,0.139232
1,1533,0.011526
2,1534,0.139848
3,1535,0.156550
4,1536,0.263580
...,...,...
165,1697,0.018605
166,1698,0.045921
167,1699,0.075831
168,1700,0.000221


In [39]:
start_time = datetime.now()
print("开始时间为:", start_time)

# 定义LightGBM回归器
lgb_regressor = lgb.LGBMRegressor(force_col_wise=True)

# 定义超参数空间
param_grid = {
    'num_leaves': [63], 
    'max_depth': [10],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.03, 0.05, 0.1]
}

# 定义网格搜索对象
lgb_filled_grid_search = GridSearchCV(estimator=lgb_regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# 进行超参数搜索
lgb_filled_model = lgb_filled_grid_search.fit(X_train_filled_cleaned.iloc[:, 2:], y_train)

# 输出最佳超参数组合和得分
print("Best Parameters:", lgb_filled_grid_search.best_params_)
print("Best Score:", lgb_filled_grid_search.best_score_)

# 在验证集上评估模型
lgb_filled_rankic, lgb_filled_valid_df = model_function.evaluate(lgb_filled_model, X_valid_filled_cleaned, y_valid)

# 预测测试集
lgb_filled_test_result = model_function.perform_pred(lgb_filled_model, X_test_filled_cleaned)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-11 20:44:17.213371
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.018577
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.015366
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.011142
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 19
[LightGBM] [Info] Start training from score -0.013043
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 3257532, number of used features: 19
[LightGBM] [Info] Start training from score -0.016644
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info]

In [40]:
with open('./data/lgb_filled.pkl', 'wb') as f:
    pickle.dump(lgb_filled_model, f)
with open('./data/lgb_filled_test_result.pkl', 'wb') as f:
    pickle.dump(lgb_filled_test_result, f)

In [41]:
lgb_filled_rankic

,date,RankIC
0,1532,0.151926
1,1533,0.018393
2,1534,0.153308
3,1535,0.155959
4,1536,0.254208
...,...,...
165,1697,0.022875
166,1698,0.045988
167,1699,0.076332
168,1700,0.002724


In [42]:
start_time = datetime.now()
print("开始时间为:", start_time)

# 定义LightGBM回归器
lgb_regressor = lgb.LGBMRegressor(force_col_wise=True)

# 定义超参数空间
param_grid = {
    'num_leaves': [63], 
    'max_depth': [10],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.03, 0.05, 0.1]
}

# 定义网格搜索对象
lgb_selected_grid_search = GridSearchCV(estimator=lgb_regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# 进行超参数搜索
lgb_selected_model = lgb_selected_grid_search.fit(X_train_filled_selected.iloc[:, 2:], y_train)

# 输出最佳超参数组合和得分
print("Best Parameters:", lgb_selected_grid_search.best_params_)
print("Best Score:", lgb_selected_grid_search.best_score_)

# 在验证集上评估模型
lgb_selected_rankic, lgb_selected_valid_df = model_function.evaluate(lgb_selected_model, X_valid_filled_selected, y_valid)

# 预测测试集
lgb_selected_test_result = model_function.perform_pred(lgb_selected_model, X_test_filled_selected)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-11 20:57:33.896391
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 10
[LightGBM] [Info] Start training from score -0.018577
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 10
[LightGBM] [Info] Start training from score -0.015366
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 10
[LightGBM] [Info] Start training from score -0.011142
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 3257531, number of used features: 10
[LightGBM] [Info] Start training from score -0.013043
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 3257532, number of used features: 10
[LightGBM] [Info] Start training from score -0.016644
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info]

In [43]:
with open('./data/lgb_selected.pkl', 'wb') as f:
    pickle.dump(lgb_selected_model, f)
with open('./data/lgb_selected_test_result.pkl', 'wb') as f:
    pickle.dump(lgb_selected_test_result, f)

In [44]:
lgb_selected_rankic

,date,RankIC
0,1532,0.121409
1,1533,0.000947
2,1534,0.155346
3,1535,0.137966
4,1536,0.252470
...,...,...
165,1697,0.031000
166,1698,0.048724
167,1699,0.129882
168,1700,0.011097


三个模型结果比较显示，未填充缺失值的模型训练得到的模型所得的RankIC均值最高，达到0.093.超过了Lasso和SVR。

尝试遗传编程回归。

In [15]:
def gp_fit(df_train, label):
    init_function = ('add', 'sub', 'mul', 'div', 'sqrt', 'log', 'abs', 'neg', 'inv', 'max', 'min', 'sin', 'cos', 'tan')
    regressor = genetic.SymbolicRegressor(
                                        population_size=1000, 
                                        metric='spearman',
                                        generations=3,
                                        tournament_size=20,
                                        parsimony_coefficient=0.0005,
                                        p_crossover=0.7,
                                        p_subtree_mutation=0.05,
                                        p_hoist_mutation=0.05,
                                        p_point_mutation=0.05,
                                        p_point_replace=0.05,
                                        function_set=init_function,
                                        max_samples=0.9,
                                        verbose=1,
                                        const_range=(-1, 1),
                                        feature_names=list(df_train.columns),
                                        n_jobs=10,
                                        random_state=int(time.time())
                                       )

    regressor.fit(df_train, label)
    print(regressor)
    print(regressor._program.raw_fitness_)
    return regressor

In [16]:
start_time = datetime.now()
print("开始时间为:", start_time)

# 创建 遗传编程回归 模型 并训练
SymbolicRegressor_model = gp_fit(X_train_filled_cleaned.iloc[:, 2:], y_train)

# 在验证集上评估模型
SymbolicRegressor_rankic, SymbolicRegressor_valid_df = model_function.evaluate(SymbolicRegressor_model, X_valid_filled_cleaned, y_valid)

# 预测测试集
SymbolicRegressor_test_result = model_function.perform_pred(SymbolicRegressor_model, X_test_filled_cleaned)

end_time = datetime.now()
print("结束时间为:", end_time)
print("处理时间为：", end_time - start_time)

开始时间为: 2024-04-16 16:58:48.369975
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.06        0.0128924        2        0.0809072        0.0800289     54.35m
neg(f_13)
0.0809071844145948
rankic均值为： 0.05994197943200105
mse on validation: 1.696949045725676
结束时间为: 2024-04-16 17:26:00.884255
处理时间为： 0:27:12.514280


In [17]:
with open('./data/SymbolicRegressor.pkl', 'wb') as f:
    pickle.dump(SymbolicRegressor_model, f)
with open('./data/SymbolicRegressor_test_result.pkl', 'wb') as f:
    pickle.dump(SymbolicRegressor_test_result, f)

In [18]:
SymbolicRegressor_rankic

,date,RankIC
0,1532,0.164501
1,1533,0.203799
2,1534,0.249424
3,1535,0.274876
4,1536,0.193630
...,...,...
165,1697,0.084067
166,1698,-0.028245
167,1699,-0.051264
168,1700,-0.044887
